In [1]:
import yfinance as yf
import pandas as pd
import threading
import numpy as np

In [2]:
# Fetching DAX tickers from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/DAX')[4]
daxTickers = list(table['Ticker'])
daxTickers[0] # adidas (ADS.DE)

'ADS.DE'

In [3]:
# Function to get history for a given ticker
def get_history(ticker, startDate = '2023-01-01', endDate = '2023-12-31'):
    df = yf.Ticker(ticker).history(start = startDate, end = endDate)
    df = df.drop(columns = ['Dividends', 'Volume', 'Stock Splits'])
    return df

In [4]:
# Function to fetch history for each ticker using threading
def get_histories(tickers, start_date='2023-01-01', end_date='2023-12-31'):
    dfs = []
    def worker(ticker):
        df = get_history(ticker, start_date, end_date)
        dfs.append((ticker,df))

    threads = [threading.Thread(target=worker, args=(ticker,)) for ticker in tickers]
    [thread.start() for thread in threads]
    [thread.join() for thread in threads]
    
    return dict(dfs)

In [5]:
data = get_histories(daxTickers)

In [23]:
def generateAlphaSignals(index, dfs = data, selection = 5):
    alphaSig = {}
    for k in dfs:
        #defining our formula
        alphaSig[k] = (
            (dfs[k].iloc[index]["Close"] - dfs[k].iloc[index]["Open"]) /
            ((dfs[k].iloc[index]["High"] - dfs[k].iloc[index]["Low"]) + 0.001)
        )
    alphaSig = sorted(alphaSig.items(), key=lambda item: item[1])
    longSig = alphaSig[len(alphaSig)-selection:]
    shortSig = alphaSig[:selection]
    return longSig,shortSig

        
generateAlphaSignals(0)

([('BAYN.DE', 0.8789268720505372),
  ('MBG.DE', 0.9140758868564579),
  ('BMW.DE', 0.9478346489909149),
  ('RHM.DE', 0.9582420168150678),
  ('CON.DE', 0.9833300667677126)],
 [('SRT3.DE', -0.9873004698453782),
  ('SHL.DE', -0.8395177840526918),
  ('ENR.DE', -0.733726719778024),
  ('QIA.DE', -0.6218656628916495),
  ('MRK.DE', -0.5866998103168953)])

In [24]:

def backTest(capital = 10_000, dfs = data):
    min_days = np.inf
    for k in dfs:
        if len(dfs[k]) < min_days:
            min_days = len(dfs[k])
    performance = [capital]
    for i in range(min_days-1):
        signals = generateAlphaSignals(i)
        for long in signals[0]:
            #we take position just before close
            capital = capital * (1 +  ((dfs[long[0]].iloc[i+1]['Close'] - dfs[long[0]].iloc[i]['Close']) / dfs[long[0]].iloc[i]['Close']))

        for short in signals[1]:
            capital = capital * (1 +  ((dfs[short[0]].iloc[i]['Close'] - dfs[short[0]].iloc[i+1]['Close']) / dfs[short[0]].iloc[i]['Close']))
        performance.append(capital)
            
        
    return capital, performance
bt = backTest()